In [1]:
import os

In [2]:
%pwd

'c:\\Users\\VENKA\\Desktop\\Data Science\\Project\\surface_crack\\Surface-Crack-Detection\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\VENKA\\Desktop\\Data Science\\Project\\surface_crack\\Surface-Crack-Detection'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config 
    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "images")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config
    

In [8]:
import time




In [9]:
    

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
      
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            str(self.config.tensorboard_root_log_dir),  # Convert WindowsPath to string
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    def _create_ckpt_callbacks(self):
        checkpoint_dir_str = str(self.config.checkpoint_model_filepath)  # Convert WindowsPath to string
        if checkpoint_dir_str.endswith('.h5'):
            return tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_dir_str,
                save_best_only=True
            )
        else:
            raise ValueError("Invalid checkpoint model filepath")

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks(),
            self._create_ckpt_callbacks()
        ]


In [10]:
import os 
import urllib.request as request
import zipfile as zipfile
import tensorflow as tf
import time

In [11]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_generator(self):
        
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator
            
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self,callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )
        
        self.save_model(
            path = self.config.trained_model_path,
            model = self.model
        )

In [14]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    training = Training(config= training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e

[2024-03-17 14:17:47,042: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-17 14:17:47,054: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-17 14:17:47,061: INFO: common: created directory at: artifacts]
[2024-03-17 14:17:47,064: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-03-17 14:17:47,068: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-03-17 14:17:47,077: INFO: common: created directory at: artifacts\training]


Found 40 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Epoch 1/20
10/10 [==============================] - 23s 2s/step - loss: 15.3687 - accuracy: 0.4875 - val_loss: 37.9038 - val_accuracy: 0.3750
Epoch 2/20
10/10 [==============================] - 20s 2s/step - loss: 19.2018 - accuracy: 0.4750 - val_loss: 35.4757 - val_accuracy: 0.3750
Epoch 3/20
10/10 [==============================] - 22s 2s/step - loss: 14.3417 - accuracy: 0.5562 - val_loss: 18.5726 - val_accuracy: 0.6250
Epoch 4/20
10/10 [==============================] - 22s 2s/step - loss: 18.2624 - accuracy: 0.4563 - val_loss: 16.0885 - val_accuracy: 0.6250
Epoch 5/20
10/10 [==============================] - 22s 2s/step - loss: 14.3123 - accuracy: 0.4938 - val_loss: 19.9278 - val_accuracy: 0.6250
Epoch 6/20
10/10 [==============================] - 21s 2s/step - loss: 10.0917 - accuracy: 0.6250 - val_loss: 11.6755 - val_accuracy: 0.6250
Epoch 7/20
10/10 [==============================] - 22s 2s/step - l